In [1]:
import torch
import torch.nn as nn

In [2]:
class conv_block(nn.Module):
    def __init__(self, in_ch, out_ch, **kwargs):
        super(conv_block, self).__init__()

        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_ch, out_ch, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_ch)

    def forward(self, x):
        x = self.conv(x)
        x = self.batchnorm(x)
        x = self.relu(x)

        return x

In [3]:
class inception_block(nn.Module):
    def __init__(self, in_ch, ch1x1, ch3x3_red, ch3x3, ch5x5_red, ch5x5, pool):
        super(inception_block, self).__init__()

        self.branch_1 = nn.Conv2d(in_ch, ch1x1, kernel_size=(1,1))

        self.branch_2 = nn.Sequential(
            conv_block(in_ch, ch3x3_red, kernel_size=(1,1)),
            conv_block(ch3x3_red, ch3x3, kernel_size=(3,3), padding=1)
        )

        self.branch_3 = nn.Sequential(
            conv_block(in_ch, ch5x5_red, kernel_size=(1,1)),
            conv_block(ch5x5_red, ch5x5, kernel_size=(5,5), padding=2)
        )

        self.branch_4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=(3,3), stride=1, padding=1),
            conv_block(in_ch, pool, kernel_size=(1,1))
        )


    def forward(self, x):
        x1 = self.branch_1(x)
        x2 = self.branch_2(x)
        x3 = self.branch_3(x)
        x4 = self.branch_4(x)

        return torch.cat([x1, x2, x3, x4], 1)

In [4]:
class GoogLeNet(nn.Module):
    def __init__(self, in_ch=3, num_classes=1000):
        super(GoogLeNet, self).__init__()

        self.conv1 = conv_block(in_ch=in_ch, out_ch=64, kernel_size=(7,7), stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.conv2 = conv_block(64, 192, kernel_size=3, stride=1, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception_3a = inception_block(192, 64, 96, 128, 16, 32, 32)
        self.inception_3b = inception_block(256, 128, 128, 192, 32, 96, 64)
        self.maxpool_3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception_4a = inception_block(480, 192, 96, 208, 16, 48, 64)
        self.inception_4b = inception_block(512, 160, 112, 224, 24, 64, 64)
        self.inception_4c = inception_block(512, 128, 128, 256, 24, 64, 64)
        self.inception_4d = inception_block(512, 112, 144, 288, 32, 64, 64)
        self.inception_4e = inception_block(528, 256, 160, 320, 32, 128, 128)
        self.maxpool_4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception_5a = inception_block(832, 256, 160, 320, 32, 128, 128)
        self.inception_5b = inception_block(832, 384, 192, 384, 48, 128, 128)

        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.dropout = nn.Dropout2d(p=0.4)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)

        x = self.inception_3a(x)
        x = self.inception_3b(x)
        x = self.maxpool_3(x)

        x = self.inception_4a(x)
        x = self.inception_4b(x)
        x = self.inception_4c(x)
        x = self.inception_4d(x)
        x = self.inception_4e(x)
        x = self.maxpool_4(x)

        x = self.inception_5a(x)
        x = self.inception_5b(x)
        x = self.avgpool(x)
        x = x.view(x.shape[0], -1)
        x = self.dropout(x)
        x = self.fc(x)

        return x

# 여기까지가 보조분류기가 없는 GoogLeNet

In [5]:
#gooLeNet의 inception모듈 만들기

class Inception_module(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3_red, ch3x3, ch5x5_red, ch5x5, pool):
        super(Inception_module, self).__init__()

        #1번째 네트워크
        self.conv1x1 = nn.Conv2d(in_channels, ch1x1, kernel_size=1)

        #2번째 네트워크
        self.conv3x3 = nn.Sequential(
            nn.Conv2d(in_channels, ch3x3_red, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch3x3_red, ch3x3, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

        #3번째 네트워크
        self.conv5x5 = nn.Sequential(
            nn.Conv2d(in_channels, ch5x5_red, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch5x5_red, ch5x5, kernel_size=5, padding=2),
            nn.ReLU(inplace=True)
        )

        #4번째 네트워크
        self.pool_net = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, pool, kernel_size=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x1 = self.conv1x1(x)
        x2 = self.conv3x3(x)
        x3 = self.conv5x5(x)
        x4 = self.pool_net(x)

        return torch.cat([x1, x2, x3, x4], 1)


In [6]:
#이거는 GoogLeNet설계하면서 필요한 네트워크 블록임.. 이거는 뭔지 잘 모르겟음..
#보조분류기 라는 거 같은데 모르겟음?

class AuxModule(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000):
        super(AuxModule, self).__init__()

        self.avgpool = nn.AdaptiveAvgPool2d((4,4))

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, 128, kernel_size=1),
            nn.ReLU(inplace=True)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(4*4*128, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.7),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.avgpool(x)
        x = self.conv1(x)
        x = torch.flatten(x, 1)
        return self.fc(x)

In [7]:
#위 설계한 inception모듈을 받아서 전체 GoogLeNet설계하기


class GoogLeNet_withAux(torch.nn.Module):
    def __init__(self, in_channels=3, num_classes=1000):
        super(GoogLeNet_withAux, self).__init__()

        self.training = True
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.LocalResponseNorm(2)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(2),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.inception_3a = Inception_module(192, 64, 96, 128, 16, 32, 32)
        self.inception_3b = Inception_module(256, 128, 128, 192, 32, 96, 64)
        self.maxpool_3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception_4a = Inception_module(480, 192, 96, 208, 16, 48, 64)
        self.aux1 = AuxModule(512, num_classes)

        self.inception_4b = Inception_module(512, 160, 112, 224, 24, 64, 64)
        self.inception_4c = Inception_module(512, 128, 128, 256, 24, 64, 64)
        self.inception_4d = Inception_module(512, 112, 144, 288, 32, 64, 64)
        self.aux2 = AuxModule(528, num_classes)

        self.inception_4e = Inception_module(528, 256, 160, 320, 32, 128, 128)
        self.maxpool_4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception_5a = Inception_module(832, 256, 160, 320, 32, 128, 128)
        self.inception_5b = Inception_module(832, 384, 192, 384, 48, 128, 128)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.dropout = nn.Dropout2d(p=0.4)
        self.fc = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        
        x = self.inception_3a(x)
        x = self.inception_3b(x)
        x = self.maxpool_3(x)

        x = self.inception_4a(x)
        if self.training:
            out1 = self.aux1(x)

        x = self.inception_4b(x)
        x = self.inception_4c(x)
        x = self.inception_4d(x)
        if self.training:
            out2 = self.aux2(x)

        x = self.inception_4e(x)
        x = self.maxpool_4(x)

        x = self.inception_5a(x)
        x = self.inception_5b(x)

        x = self.avgpool(x)

        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)                
        if self.training:
            return [x, out1, out2]
        else:
            return x
    
    # 훈련 모드 설정
    def set_train(self):
        self.train()  # 내장 train 메서드를 호출

    # 평가 모드 설정
    def set_eval(self):
        self.eval()  # 내장 eval 메서드를 호출

In [8]:
import torchsummary

debug_model = GoogLeNet()
torchsummary.summary(debug_model, input_size=(3, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
        conv_block-4         [-1, 64, 112, 112]               0
         MaxPool2d-5           [-1, 64, 56, 56]               0
            Conv2d-6          [-1, 192, 56, 56]         110,784
       BatchNorm2d-7          [-1, 192, 56, 56]             384
              ReLU-8          [-1, 192, 56, 56]               0
        conv_block-9          [-1, 192, 56, 56]               0
        MaxPool2d-10          [-1, 192, 28, 28]               0
           Conv2d-11           [-1, 64, 28, 28]          12,352
           Conv2d-12           [-1, 96, 28, 28]          18,528
      BatchNorm2d-13           [-1, 96, 28, 28]             192
             ReLU-14           [-1, 96,

c:\Users\Z13_ASH\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


In [9]:
debug_model_2 = GoogLeNet_withAux()
torchsummary.summary(debug_model_2, input_size=(3, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
              ReLU-2         [-1, 64, 112, 112]               0
         MaxPool2d-3           [-1, 64, 56, 56]               0
 LocalResponseNorm-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,160
              ReLU-6           [-1, 64, 56, 56]               0
            Conv2d-7          [-1, 192, 56, 56]         110,784
              ReLU-8          [-1, 192, 56, 56]               0
 LocalResponseNorm-9          [-1, 192, 56, 56]               0
        MaxPool2d-10          [-1, 192, 28, 28]               0
           Conv2d-11           [-1, 64, 28, 28]          12,352
           Conv2d-12           [-1, 96, 28, 28]          18,528
             ReLU-13           [-1, 96, 28, 28]               0
           Conv2d-14          [-1, 128,

# 여기까지가 보조분류기(AuxModule) 있는 GoogLeNet

## 여기는 공통의 데이터 전처리 부분

In [10]:
import torchvision #예제 이미지 데이터셋이 모여있는 모듈
from torchvision import datasets, transforms

In [11]:
train_raw_data = datasets.CIFAR100("raw_data", 
                                   train=True, 
                                   download=True, 
                                   transform=transforms.ToTensor())
test_raw_data = datasets.CIFAR100("raw_data", 
                                   train=False, 
                                   download=True, 
                                   transform=transforms.ToTensor())

100%|██████████| 169001437/169001437 [00:45<00:00, 3715643.78it/s]


Extracting raw_data\cifar-100-python.tar.gz to raw_data
Files already downloaded and verified


In [12]:
import numpy as np

#이미지 전처리11 : RGB채널별 평균/표준편차 계산 함수 설계
def normal_parm_func(input_data):

    meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x, _ in input_data]
    stdRGB = [np.std(x.numpy(), axis=(1,2)) for x, _ in input_data]

    RGB_mean_list, RGB_std_list = [], []

    for i in range(3):
        RGB_mean_list.append(
            np.mean([m[i] for m in meanRGB])
        )
        RGB_std_list.append(
            np.mean([s[i] for s in stdRGB])
        )

    return RGB_mean_list, RGB_std_list


proc_train_data = normal_parm_func(train_raw_data)
proc_test_data = normal_parm_func(test_raw_data)

print(f"Train Mean: {[format(m, '.3f') for m in proc_train_data[0]]}, "
      f"Train Std: {[format(s, '.3f') for s in proc_train_data[1]]}")
print(f"Test Mean: {[format(m, '.3f') for m in proc_test_data[0]]}, "
      f"Test Std: {[format(s, '.3f') for s in proc_test_data[1]]}")

Train Mean: ['0.507', '0.487', '0.441'], Train Std: ['0.201', '0.198', '0.202']
Test Mean: ['0.509', '0.487', '0.442'], Test Std: ['0.202', '0.200', '0.204']


In [13]:
#이미지 변환방식이 저장된 객체 생성
train_transformation = transforms.Compose([
    transforms.ToTensor(), 
    #입력하려는 train_raw데이터가 텐서 자료형이지만
    #이걸 안써주면 PIL자료형이 되버림...
    transforms.Resize((224, 224)), #GooGleNet 입력 이미지는 224 224 3임
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean=proc_train_data[0], std=proc_train_data[1])
    #훈련데이터의 평균, 표준편차 입력
])
test_transformation = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)), #GooGleNet 입력 이미지는 224 224 3임
    transforms.Normalize(mean=proc_train_data[0], std=proc_train_data[1])
    #훈련데이터의 평균, 표준편차 입력
])

#이미지에 위 변환방식을 적용
train_raw_data.transform = train_transformation
test_raw_data.transform = test_transformation

#데이터 로더 생성 -> 여기서 Batch_size를 정의함
train_loader = torch.utils.data.DataLoader(train_raw_data, batch_size=48, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_raw_data, batch_size=48, shuffle=False)

In [14]:
def count_images_and_labels(loader):
    num_images = 0
    label_set = set()
    for data, labels in loader:
        num_images += data.size(0)
        label_set.update(labels.tolist())
    
    return num_images, len(label_set)


train_images, train_label_count = count_images_and_labels(train_loader)
test_images, test_label_count = count_images_and_labels(test_loader)

print(f"Training Images: {train_images}, Unique Labels: {train_label_count}")
print(f"Testing Images: {test_images}, Unique Labels: {test_label_count}")

Training Images: 50000, Unique Labels: 100
Testing Images: 10000, Unique Labels: 100


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## 보조분류기 없는 GoogLeNet먼저 수행

In [16]:
ex_model = GoogLeNet(in_ch= 3, num_classes=train_label_count)
ex_model.to(device)

GoogLeNet(
  (conv1): conv_block(
    (relu): ReLU()
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2): conv_block(
    (relu): ReLU()
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (inception_3a): inception_block(
    (branch_1): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
    (branch_2): Sequential(
      (0): conv_block(
        (relu): ReLU()
        (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
        (batchnorm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): conv_

In [17]:
learning_rate = 0.0002

import torch.nn.functional as F #이거는 활성화 함수 모듈
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ex_model.parameters(), lr=learning_rate)

In [18]:
#훈련을 위한 함수 정의
from tqdm import tqdm #훈련 진행상황 체크

def model_train(model, data_loader, loss_fn, optimizer_fn, processing_device):

    model.train() #모델을 훈련 모드로 설정

    #loss와 accuracy를 계산하기 위한 임시 변수를 생성
    run_size, run_loss, corr = 0, 0, 0

    progress_bar = tqdm(data_loader) #이때 사용되는 데이터는 훈련용 데이터


    for batch_idx, (image, label) in enumerate(progress_bar, start=1):
        #입력된 데이터를 먼저 GPU로 이전하기
        image = image.to(processing_device)
        label = label.to(processing_device)

        #전사과정 수행
        output = model(image)

        loss = loss_fn(output, label)

        #옵티마이저의 Gradient 초기화
        optimizer_fn.zero_grad()

        #backward 과정 수행
        loss.backward() #Backprogration을 진행하여 Gradient계산
        optimizer_fn.step() #계산된 gradient(모델 파라미터)를 업데이트


        #여기부터는 학습이 잘 되고 있는지 확인하는 부분
        _, pred = output.max(dim=1)
        corr += pred.eq(label).sum().item()
        
        run_loss += loss.item() * image.size(0)
        run_size += image.size(0)
        progress_bar.set_description('[Training] loss: ' + \
                            f'{run_loss / run_size:.4f}, accuracy: ' + \
                            f'{corr / run_size:.4f}')
        
    acc = corr / len(data_loader.dataset)

    return run_loss / len(data_loader.dataset), acc

In [19]:
#평가를 위한 구문 작성
def model_evaluate(model, data_loader, loss_fn, processing_device):
    model.eval() #모델을 평가 모드로 전환

    #gradient업데이틀를 방지해주자
    with torch.no_grad():

        #여기서도 loss, accuracy 계산을 위한 임시 변수 선언
        run_loss, corr = 0, 0


        for image, label in data_loader: #이때 사용되는 데이터는 평가용 데이터
            #입력된 데이터를 먼저 GPU로 이전하기
            image = image.to(processing_device)
            label = label.to(processing_device)


            #평가 결과를 도출하자
            outputs = model(image) #평가모드는 1개만 출력이 나오나보네...

            _, pred = outputs.max(dim=1)

            
            #모델의 평가 결과 도출 부분
            # 배치의 실제 크기에 맞추어 정확도와 손실을 계산
            corr += torch.sum(pred.eq(label)).item()
            run_loss += loss_fn(outputs, label).item() * image.size(0)

        # 전체 데이터셋에 대한 평균 손실과 정확도 계산
        acc = corr / len(data_loader.dataset)

        return run_loss / len(data_loader.dataset), acc

In [20]:
min_loss = np.inf
MODEL_NAME = 'GoogLeNet'


for epoch in range(100):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    train_loss, train_acc = model_train(ex_model, train_loader, criterion, optimizer, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate(ex_model, test_loader, criterion, device)   
    
    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from '
            f'{min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(ex_model.state_dict(), f'{MODEL_NAME}.pth')
    
    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, '
        f'val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

  0%|          | 0/1042 [00:00<?, ?it/s]c:\Users\Z13_ASH\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
c:\Users\Z13_ASH\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
[

[INFO] val_loss has been improved from inf to 3.26544. Saving Model!
epoch 01, loss: 3.63296, acc: 0.13622, val_loss: 3.26544, val_accuracy: 0.19810


[Training] loss: 2.7543, accuracy: 0.2873: 100%|██████████| 1042/1042 [02:11<00:00,  7.95it/s]


[INFO] val_loss has been improved from 3.26544 to 2.40563. Saving Model!
epoch 02, loss: 2.75427, acc: 0.28726, val_loss: 2.40563, val_accuracy: 0.36570


[Training] loss: 2.1960, accuracy: 0.4040: 100%|██████████| 1042/1042 [02:10<00:00,  7.97it/s]


...

epoch 98, loss: 0.04971, acc: 0.98396, val_loss: 2.05051, val_accuracy: 0.68960


[Training] loss: 0.0612, accuracy: 0.9802: 100%|██████████| 1042/1042 [02:10<00:00,  7.96it/s]


epoch 99, loss: 0.06117, acc: 0.98016, val_loss: 2.05720, val_accuracy: 0.69200


[Training] loss: 0.0553, accuracy: 0.9820: 100%|██████████| 1042/1042 [02:12<00:00,  7.84it/s]


epoch 100, loss: 0.05534, acc: 0.98198, val_loss: 2.10171, val_accuracy: 0.69350


In [21]:
ex_model.load_state_dict(torch.load(f'{MODEL_NAME}.pth'))

final_loss, final_acc = model_evaluate(ex_model, test_loader, criterion, device)
print(f'evaluation loss: {final_loss:.5f}, evaluation accuracy: {final_acc:.5f}')

evaluation loss: 1.25536, evaluation accuracy: 0.65640


# 보조분류기 있는 GoogLeNet 수행

In [22]:
ex_model_1 = GoogLeNet_withAux(in_channels=3, num_classes=train_label_count)
ex_model_1.to(device)

GoogLeNet_withAux(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (3): LocalResponseNorm(2, alpha=0.0001, beta=0.75, k=1.0)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): LocalResponseNorm(2, alpha=0.0001, beta=0.75, k=1.0)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (inception_3a): Inception_module(
    (conv1x1): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv3x3): Sequential(
      (0): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
    )
    (co

In [23]:
learning_rate = 0.0002

import torch.nn.functional as F #이거는 활성화 함수 모듈
import torch.optim as optim

criterion_1 = nn.CrossEntropyLoss()
optimizer_1 = optim.Adam(ex_model_1.parameters(), lr=learning_rate)

In [24]:
from tqdm import tqdm #훈련 진행상황 체크

def model_train_withAux(model, data_loader, loss_fn, optimizer_fn, processing_device):

    model.train() #모델을 훈련 모드로 설정

    #loss와 accuracy를 계산하기 위한 임시 변수를 생성
    run_size, run_loss, corr = 0, 0, 0

    progress_bar = tqdm(data_loader) #이때 사용되는 데이터는 훈련용 데이터


    for batch_idx, (image, label) in enumerate(progress_bar, start=1):
        #입력된 데이터를 먼저 GPU로 이전하기
        image = image.to(processing_device)
        label = label.to(processing_device)

        #전사과정 수행
        pred0, pred1, pred2 = model(image) #출력이 3개 나온다.

        loss_0 = loss_fn(pred0, label)
        loss_1 = loss_fn(pred1, label)
        loss_2 = loss_fn(pred2, label) #출력별 loos중간결과값을 계산함
        
        loss = loss_0 + 0.3*(loss_1 + loss_2)

        #옵티마이저의 Gradient 초기화
        optimizer_fn.zero_grad()

        #backward 과정 수행
        loss.backward() #Backprogration을 진행하여 Gradient계산
        optimizer_fn.step() #계산된 gradient(모델 파라미터)를 업데이트


        #여기부터는 학습이 잘 되고 있는지 확인하는 부분
        _, pred = pred0.max(dim=1)
        corr += pred.eq(label).sum().item()
        
        run_loss += loss.item() * image.size(0)
        run_size += image.size(0)
        progress_bar.set_description('[Training] loss: ' + \
                            f'{run_loss / run_size:.4f}, accuracy: ' + \
                            f'{corr / run_size:.4f}')
        
    acc = corr / len(data_loader.dataset)

    return run_loss / len(data_loader.dataset), acc

In [25]:
#평가를 위한 구문 작성
def model_evaluate_withAux(model, data_loader, loss_fn, processing_device):
    model.eval() #모델을 평가 모드로 전환

    #gradient업데이틀를 방지해주자
    with torch.no_grad():

        #여기서도 loss, accuracy 계산을 위한 임시 변수 선언
        run_loss, corr = 0, 0


        for image, label in data_loader: #이때 사용되는 데이터는 평가용 데이터
            #입력된 데이터를 먼저 GPU로 이전하기
            image = image.to(processing_device)
            label = label.to(processing_device)


            #평가 결과를 도출하자
            outputs = model(image) #평가모드는 1개만 출력이 나오나보네...

            _, pred = outputs.max(dim=1)

            
            #모델의 평가 결과 도출 부분
            # 배치의 실제 크기에 맞추어 정확도와 손실을 계산
            corr += torch.sum(pred.eq(label)).item()
            run_loss += loss_fn(outputs, label).item() * image.size(0)

        # 전체 데이터셋에 대한 평균 손실과 정확도 계산
        acc = corr / len(data_loader.dataset)

        return run_loss / len(data_loader.dataset), acc

In [26]:
min_loss = np.inf
MODEL_NAME = 'GoogLeNet_withAuX'


for epoch in range(100):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    train_loss, train_acc = model_train_withAux(ex_model_1, train_loader, criterion_1, optimizer_1, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate_withAux(ex_model_1, test_loader, criterion_1, device)   
    
    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from '
            f'{min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(ex_model_1.state_dict(), f'{MODEL_NAME}.pth')
    
    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, '
        f'val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

[Training] loss: 6.6673, accuracy: 0.0532: 100%|██████████| 1042/1042 [02:35<00:00,  6.69it/s]


[INFO] val_loss has been improved from inf to 3.74917. Saving Model!
epoch 01, loss: 6.66730, acc: 0.05322, val_loss: 3.74917, val_accuracy: 0.11400


[Training] loss: 5.8225, accuracy: 0.1395: 100%|██████████| 1042/1042 [02:26<00:00,  7.13it/s]


[INFO] val_loss has been improved from 3.74917 to 3.38131. Saving Model!
epoch 02, loss: 5.82252, acc: 0.13946, val_loss: 3.38131, val_accuracy: 0.18080


[Training] loss: 5.2532, accuracy: 0.2072: 100%|██████████| 1042/1042 [02:30<00:00,  6.93it/s]


...

epoch 98, loss: 0.42532, acc: 0.97214, val_loss: 3.24920, val_accuracy: 0.59730


[Training] loss: 0.4121, accuracy: 0.9741: 100%|██████████| 1042/1042 [03:07<00:00,  5.57it/s]


epoch 99, loss: 0.41213, acc: 0.97412, val_loss: 3.08959, val_accuracy: 0.60170


[Training] loss: 0.3982, accuracy: 0.9771: 100%|██████████| 1042/1042 [03:08<00:00,  5.53it/s]


epoch 100, loss: 0.39819, acc: 0.97708, val_loss: 3.56683, val_accuracy: 0.58290


In [27]:
ex_model_1.load_state_dict(torch.load(f'{MODEL_NAME}.pth'))

final_loss, final_acc = model_evaluate(ex_model_1, test_loader, criterion_1, device)
print(f'evaluation loss: {final_loss:.5f}, evaluation accuracy: {final_acc:.5f}')

evaluation loss: 1.63855, evaluation accuracy: 0.56660
